@Data mining

# Data Preparation

In this first task, you will **examine all data attributes and identify issues present in the data**. For each of the issues that you have identified, choose and perform necessary actions to address it. 

Finally, you will need to suitably split the data into two sets: one for training and one for testing, the latter contains 100 samples with missing class labels. Your marks for this task will depend on how well you identify the issues and address them. Below is a list of data preparation issues that you need to address

    • Identify and remove irrelevant attributes.
        - Att09 has a lot of missing values so maybe remove it
    • Detect and handle missing entries
        - Att00 has been replaced by the mean value for now
    • Detect and handle duplicates (both instances and attributes).
    • Select suitable data types for attributes.
    • Perform data transformation (such as scaling/standardization) if needed.
    • Perform other data preparation operations (This is optional, bonus marks will be awarded for novel ideas).

In [ ]:
DATA = 'https://github.com/kituyiharry/ClassData/raw/master/Assignment2021.sqlite'
FNAME= 'Assignment2021.sqlite'
import urllib.request


print("================== Fetching data")

urllib.request.urlretrieve(DATA, FNAME)

#### Data Exploration

In [ ]:
import pandas as pd
import sqlite3

Connect to the database via SQLIte and load into pandas dataframe
* There are 1200 entries, the last 200 are missing attributes
* _class_ column is what we are trying to predict
* Some attributes are numeric, some categorical

From Assignment description:
The data is known to contain imperfections:
 * There are missing/corrupted entries in the data set.
 * There are duplicates, both instances and attributes.
 * There are irrelevant attributes that do not contain any useful information useful for the classification task.
 * The labelled data is imbalanced: there is a considerable difference between the number of samples from each class.

In [ ]:
con = sqlite3.connect("Assignment2021.sqlite")
df  = pd.read_sql("SELECT * from data;", con)
df.head()

In [ ]:
%matplotlib inline

In [ ]:
df[:1000].describe()

In [ ]:
# pd.options.display.float_format = "{:.2f}".format

#NB: Att01 is categorical
df[:1000][['Att0'+str(n) for n in range(0,10) ]].describe()

Check column by column

In [ ]:
df[:1000][['Att1'+str(n) for n in range(0,10) ]].describe()

In [ ]:
df[:1000][['Att2'+str(n) for n in range(0,9) ]].describe()

Att09 has a lot of missing entries lets confirm

Lets filter attributes with missing attributes, Display percentage of entries missing

In [ ]:
list(filter(lambda f: f[1] > 0, map(lambda g: (g, (sum(df[:1000][g].isna())/len(df[:1000][g])) * 100 ) , df.columns[:-1])))

Around 60% is empty, Replace with mean value for the <1% Att00 for now. ~> 50%

In [ ]:
list(map(lambda v: df[v].fillna(value=df[v].mean(), inplace=True), ['Att00', 'Att09']))

df.drop(columns=['Att09'], inplace=True, errors='ignore')

In [ ]:
# See all types:
df.dtypes

Attributes 1, 8 and 29 have Object types and need further inspection


In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from pprint import pprint
import numpy as np
from scipy import stats

In [ ]:
# Look at the distributions
df[:1000].hist(grid=True,figsize=(16, 16))

In [ ]:
# Overview of the Data
from itertools import chain
import numpy as np
from math import sqrt, log

# Generator to create distinct items:
def uniq(iterable):
    seen = set()
    for x in iterable:
        if x in seen:
            continue
        seen.add(x)
        yield x

# NB: Consume all iterators to avoid problems

uniforms    = ['Att{}'.format(str(n).zfill(2)) for n in [ 13,20,24]]
categorical = ['Att01', 'Att08', 'Att29'] # Possibly multimodal
bimodal     = ['Att21', 'Att23']          # possibly also categorical 
target      = ['class']
irrelevant  = ['index']
sparse      = ['Att09']
gaussians   = list(filter(
    lambda x: x not in uniq(chain(bimodal, uniforms, target, categorical, sparse, irrelevant)), 
    ['Att{}'.format(str(n).zfill(2)) for n in range(0,30)]
))

In [ ]:
df[:1000].tail()

In [ ]:
# Some of the Object types
# Just for browsing some of the columns (Object types)

print("============= Categorical columns")

setA01 = set(df['Att01'])        # ->. {'ACKH', 'BYUB', 'GHKA', 'LLTF', 'LWYW', 'OSUG', 'SCIJ', 'UJJW', 'UKEV'}
setA08 = set(df['Att08'])        # ->  {'HFTX', 'YIFL'}
setA29 = set(df['Att29'])        # ->  {'FLJD', 'HUUV', 'OELG', 'OQDJ', 'PJIY', 'TOYT', 'YLWZ'}

pprint({
    'A01': setA01,
    'A08': setA08,
    'A29': setA29
})

# Binarization of these attributes before deciding what to do with them
# Convert them into Categorical types and replace them for feature engineering
wascategorical = ['Att01', 'Att08', 'Att29'] # Possibly multimodal

# For each object attribute, convert to a Categorical attribute
cats = list(map(lambda c: df[c].astype('category'), wascategorical))


df['Att01'] = cats[0] # For att01 
df['Att08'] = cats[1] # For att08
df['Att29'] = cats[2] # For att29 

df[['Att01','Att08','Att29']]

Define a function to encode columns using an encoder into values we can use in our classifier

In [ ]:
from sklearn import preprocessing

# Assumption being the information is somehow conveyed in the ordering of the name 
# therefore AAAA < ZZZZ or Vice Versa

def encodecategories(adf, column, encoder):
    encoder.fit(adf[[column]])
    tle = encoder.transform(adf[[column]])
    adf[column] = tle
    return encoder

# Encode categorical entries

att01enc = encodecategories(df, 'Att01', preprocessing.OrdinalEncoder())
att08enc = encodecategories(df, 'Att08', preprocessing.OrdinalEncoder())
att29enc = encodecategories(df, 'Att29', preprocessing.OrdinalEncoder()) 

df[categorical]

Bin our numerical features to reduce small observation errors. This can help the classifiers better grok the features

In [ ]:
# Convert all categorical attributes to numerical 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
# https://stackoverflow.com/a/32011969

# Bin remaining columns and go categorical
# Settling at 10 Bins for each of the remaining categories


le,_ = df[:1000].shape
# TAX = 5
# NUMBINS = int(sqrt(le)) - TAX # https://stackoverflow.com/a/33458879
NUMBINS = int(1 + (3.322 * log(le))) # Sturges Rule


print("=================  Numbins = {}".format(NUMBINS))

def binfeatures(adf, feature, numbins, dup='drop'):
    return (feature, pd.cut(adf[feature], numbins, retbins=True, duplicates=dup))
    

def dobinning(adf, numbins, columns):
    # plt.figure(figsize=(16, 22)) 
    # plt.tight_layout()
    # plt.axis('off')

    for i,(feature,(res,bins)) in enumerate(map(lambda c: binfeatures(adf, c, numbins), columns)):
        adf[feature] = res.astype('category')
        print("Feature: ", feature)
        print("Bins.  : ", bins)
        # res.value_counts(sort=False).plot(kind='bar')
        # ax = plt.subplot(7,4,i+1)
        # ax.set_title(feature)
        # ax.set_axis_off()

    # Encode the Bins as numerical values, check the Distributions
    print("\n================== Encoding bins using OrdinalEncoder")
    cat_columns = adf.select_dtypes(['category']).columns
    print(cat_columns)

    # learn the categories, get encoders for each category
    encs = list(map(lambda col: encodecategories(adf, col, preprocessing.OrdinalEncoder()), cat_columns))

    # le = preprocessing.OrdinalEncoder()
    # le.fit(adf[cat_columns])
    # tle = le.transform(adf[cat_columns])
    # adf[cat_columns] = tle

    # df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

    # plt.show()
    
dobinning(df, NUMBINS, list(uniq(chain(gaussians,uniforms))))

In [ ]:
# Check the Data types of All our columns
df.dtypes

Have a look at Multimodal and Bimodal distributions


In [ ]:
sdists = ['Att01', 'Att08', 'Att23', 'Att21']

df[sdists].describe() # Maybe True false values ??

Use Z-score scaling for each of the columns which is sensitive to outliers which may be present

In [ ]:
from sklearn.preprocessing import StandardScaler
# Z-Score Scaling as we aren't concerned about outliers.
toscale = ['Att{}'.format(str(n).zfill(2)) for n in range(0,30) if n != 9 ]
print(df.columns)
scaler = StandardScaler()
scaler.fit(df[toscale])
df[toscale] = scaler.transform(df[toscale])

In [ ]:
# Check duplication
df[df.duplicated() == True]

Separate our labelled and unalabelled features

In [ ]:
udf = df[1000:].copy(deep=True) # Unlabelled data
ldf =  df[:1000].copy(deep=True) # Labelled data

In [ ]:
ldf.tail()

In [ ]:
udf.head()

For each of the above issues your report should:

    * Describe the relevant issue in your own words and explain why it is important to address it. Your explanation must consider the classification task that you will undertake subsequently.
    * Demonstrate clearly that such an issue exists in the data with suitable illustration/evidence.
    * Clearly state and explain your choice of action to address such an issue.
    * Demonstrate convincingly that your action has addressed the issue satisfactorily. Where applicable, you should provide references to support your arguments

View correlation between attributes using df.corr

In [ ]:
import seaborn as sns

In [ ]:
# use seaborn to do the plot
fig, ax = plt.subplots(figsize=(22,22))         # Sample figsize in inches

corrs = df.corr()

# Correlation for preprocessing
sns.heatmap(corrs, annot=True, cmap=plt.cm.Reds, ax=ax)

In [ ]:
# Drop the sparse dataset and any other attributes we may not need,
# Return main features and target for feature selection

def preparefeatureselection(adf, todrop, errors='ignore'):
    pX = adf[ list(filter(lambda h: h not in todrop , ['Att{}'.format(str(n).zfill(2)) for n in range(0,30)] )) ]
    print("================= Columns used for feature selection ")
    print(pX.columns)
    print("================= Shape of remaining data ")
    print(pX.shape)
    py = adf['class']
    return pX, py

oX, y = preparefeatureselection(ldf, sparse)

###  Feature Selection

Get the best independent features for our target attribute 

In [ ]:
# https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection
from sklearn.feature_selection import  SelectPercentile, f_classif

print("Columns: ", oX.columns)
print("====== Incoming shape:  ", oX.shape)

# Cherry pick the 60th percentile by correlation based on f_classif (Numerical -> Categorical classification)
pselector = SelectPercentile(f_classif, percentile=60)
pselector.fit(oX, y)
print("====== Column selected")

# Mask against selected columns to a new DataFrame
corrstrong = list(map(lambda a : a[1] , filter(lambda z : z[0], zip(pselector.get_support(), oX.columns))))

print(corrstrong)


X_P = pselector.transform(oX) 
print("======= Outgoing shape: ", X_P.shape)


X_new = pd.DataFrame(X_P, columns=corrstrong)

X_new.head()

In [ ]:
# Check final dataframe features
X_new.describe()

Check the ratio of imbalance in our target categeory


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
from collections import Counter

# See the number of count in each class -> Noticed the class imbalance ratio is like: 2:3:5
print(Counter(y))

In [ ]:
y.hist()

Given the features in our dataset, lets find the minimum number of features that can be used to represent it, reducing the dimensionality. Let us first just check this relationship. 

This plot is just to give us an idea first for the whole dataset the test and prediction data will only be transformed and the training set will be fitted and transformed.

In [ ]:
# https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html
pca = PCA().fit(X_new)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
# DO PCA on the training and testing data
# https://stats.stackexchange.com/questions/55718/pca-and-the-train-test-split
# https://stats.stackexchange.com/a/125328
# https://stats.stackexchange.com/questions/125172/pca-on-train-and-test-datasets-should-i-run-one-pca-on-traintest-or-two-separa
# Therefore fit the PCA on the training data and transform the testing data

def fitpca(X, comp=13):
    pca = PCA(n_components=comp)
    Xout = pca.fit_transform(X)
    return Xout,pca


def transformpca(X, pca):
    X_PCA = pca.transform(X)
    print("original shape:   ", X.shape)
    print("transformed shape:", X_PCA.shape)
    return X_PCA

#### Split our Training and Testing data and Dimensionality reduction

In [ ]:
# Remember we have a class imbalance, use stratify on train test split
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/

X_train, X_test, y_train, y_test = train_test_split(X_new, y,
                                                    test_size=0.33,
                                                    random_state=2,
                                                    stratify=y   # For the class imbalance
                                                   )
# Fit PCA on our training
X_train, pca = fitpca(X_train, comp=13)

# Only transfor the test
X_test = transformpca(X_test, pca)

# Data Classification

For this task, you will demonstrate convincingly how you select, train, and fine tune your predictive
models to predict the missing labels. You must use at least the three (3) classifiers that have been
discussed in the workshops, namely k-NN, Naive Bayes, and Decision Trees. You can also select
additional classifiers (both base classifiers and meta-classifiers). 
Attempt and report the following:

• Class imbalance: the original labelled data is not equally distributed between the three classes.
You need to demonstrate that such an issue exists within the data, explain the importance of
this issue, and describe how you address this problem.

• Model training and tuning: Every classifier typically has hyperparameters to tune in order. For
each classifier, you need to select (at least one) and explain the tuning hyperparameters of your
choice. You must select and describe a suitable cross-validation/validation scheme that can
measure the performance of your model on labelled data well and can address the class
imbalance issue. Then you will need to conduct the actual tuning of your model and report the
tuning results in detail. You are expected to look at several classification performance metrics
and make comments on the classification performance of each model. Finally, you will need to
clearly indicate and justify the selected values of the tuning hyperparameters of each model.

## KNN Classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit 
from sklearn.preprocessing import OrdinalEncoder

print()
print("==================== Check the Split ratios")
print(Counter(y_train))
print(Counter(y_test))
print()

parameters = {'weights': ('uniform', 'distance'),
              'p': [1,2],
              'metric': ['euclidean', 'manhattan','minkowski'],
              'n_neighbors':[1 , 3, 5, 7, 11, 17, 21]}

# https://stackoverflow.com/questions/34731421/whats-the-difference-between-kfold-and-shufflesplit-cv
# https://xzz201920.medium.com/stratifiedkfold-v-s-kfold-v-s-stratifiedshufflesplit-ffcae5bfdf
skf = StratifiedKFold(n_splits=10,shuffle=True, random_state=2)

# sss = StratifiedShuffleSplit(n_splits=10, test_size=0.33, random_state=2)


print("==================== Fitting KNN")
knn = KNeighborsClassifier()


k_gscv = GridSearchCV(estimator=knn,
                    param_grid=parameters,
                    cv= skf, # Cross validation
                    scoring='balanced_accuracy')

results = k_gscv.fit(X_train, y_train)

print("==================== Crossvalidation results")
print(results.best_params_, results.best_score_)

fig, ax = plt.subplots(1,1, figsize=(6, 6))

print("==================== Confusion Matrix")
metrics.plot_confusion_matrix(k_gscv,
                              X_test, y_test,
                              display_labels=['0', '1', '2'],
                              ax=ax)

print("==================== Trial prediction")
y_pred = k_gscv.predict(X_test)

print("==================== Classification report")
print(metrics.classification_report(y_test, y_pred))

print("Accuracy Score = ", sep='', end='')
print(metrics.accuracy_score(y_test, y_pred))

plt.tight_layout()
plt.show()

## Decision tree classifier

In [ ]:
## Use a descision tree classifier
## Using a descision tree classifier:
## - Train the classifier using both the `Gini index` and `entropy` criterion for splitting.
## - Choose the classifier which has the highest F1 score as your best classifier.

from sklearn import tree


parameters = {'criterion': ('gini', 'entropy'),
              'min_samples_split':[3, 5, 15, 20, 25 ],
              'min_samples_leaf': [n for n in range(2,11)],
              'max_depth': [n for n in range(3,9)],
              'class_weight': [{0.0:2, 1.0:3, 2.0:5}],
              'splitter': ('best','random'),
             }

print("==================== Fitting Decision tree")
                                # Counter({2.0: 373, 1.0: 226, 0.0: 151})
dtc = tree.DecisionTreeClassifier(random_state=2)
d_gscv = GridSearchCV(estimator=dtc,
                    param_grid=parameters,
                    cv=skf,
                    scoring='balanced_accuracy')

results = d_gscv.fit(X_train, y_train)

print("==================== Crossvalidation results")
print(results.best_params_, results.best_score_)

# fig, ax = plt.subplots(1,1, figsize=(12,12))
# tree.plot_tree(
#               gscv.best_estimator_, 
#               filled=True, # color the nodes based on class/purity
#               ax=ax, fontsize=12)
# plt.show()

fig, ax = plt.subplots(1,1, figsize=(6, 6))

print("==================== Confusion matrix")
metrics.plot_confusion_matrix(d_gscv,
                              X_test, y_test,
                              display_labels=['0', '1', '2'],
                              ax=ax)

y_pred = d_gscv.predict(X_test)

print("==================== Classification report")
print(metrics.classification_report(y_test, y_pred))

print("Accuracy = ", sep='', end='')
print(metrics.accuracy_score(y_test, y_pred))

plt.tight_layout()
plt.show()

## Naive Bayes Classifier

In [ ]:
from sklearn import naive_bayes


print(X_train.shape)

# no real parameters to adjust here
fig, ax = plt.subplots(1,1)
nb = naive_bayes.GaussianNB()
classifier = nb.fit(X_train, y_train)

metrics.plot_confusion_matrix(classifier,
                              X_test, y_test,
                              display_labels=['0', '1', '2'],
                              ax=ax)

y_pred = nb.predict(X_test)

print(metrics.classification_report(y_test, y_pred))

print("Accuracy = ", sep='', end='')
print(metrics.accuracy_score(y_test, y_pred))

plt.tight_layout()
plt.show()

# Prediction

Use the best two (2) models that you have identified in the previous step to predict the
missing class labels of the last 200 samples in the original data set. Clearly explain in
detail how you arrive at the prediction.
o Provide your prediction in the report by creating a table, the first column is the sample
ID, the second and third columns are the predicted class labels respectively. Observe
and comment on the prediction that you have produced.
o Produce an sqlite3 database file with the name Answers.sqlite that contains your
prediction in the format: the first column is the sample ID, the second and third columns
are the predicted class labels. All columns should be integers. This file must be
submitted electronically with the electronic copy of the report via Blackboard. An
example of such a file is given below:

- Using accuracy and F1 score : KNN and Naive bayes

In [ ]:
# View the unlabelled entries for sparse data
# Only the predicted elements seem to be missing which is OK
udf.describe()

In [ ]:
udf.head()

In [ ]:
udf[udf.duplicated() == True]

In [ ]:
uX, uy = preparefeatureselection(udf, sparse)

In [ ]:
# Select the features used on our models ( in the 60th percentile )
# uX[corrstrong]
fX = pd.DataFrame(pselector.transform(uX), columns=corrstrong)
fX

In [ ]:
# Apply PCA on our prediction data
preddata = transformpca(fX, pca)

## KNN Classifier prediction

In [ ]:
# KNN prediction based on features
ky = k_gscv.predict(preddata)

In [ ]:
# Check the Distribution of the prediction
Counter(ky)

In [ ]:
ky

## Naive Bayes prediction

In [ ]:
# Naive bayes prediction based on features
ny = nb.predict(preddata)

In [ ]:
# Check the Distribution of the prediction
Counter(ny)

In [ ]:
pprint(ny)

our prediction distribution relatively reflects the input data in terms of distribution. It is likely that the imbalance leaked through

In [ ]:
# Prepare our dataframe for exporting
finaldf = pd.DataFrame([
    range(1000,1200), ky, ny
]).transpose()

finaldf.columns = ['index', 'Predict1', 'Predict2']

finaldf['index'] = finaldf['index'].astype(int)
finaldf['Predict1'] = finaldf['Predict1'].astype(int)
finaldf['Predict2'] = finaldf['Predict2'].astype(int)

finaldf.dtypes

In [ ]:
finaldf

In [ ]:
# Export the data
ccon = sqlite3.connect("Answers.sqlite")

finaldf.to_sql("data", ccon,  if_exists='replace', index=False) # dtype={"Predict1": Integer(), "Predict2": Integer() })